In [ ]:
import sys

!conda install --yes --prefix {sys.prefix} pytorch torchvision cpuonly -c pytorch

Pytorch Tutorial
=============

This tutorial presents Pytorch, a Python-based scientific computing package developed for deep-learning that implements automatic-differentiation in a user-friendly framework. It has also the advantage to enable computations and derivations on Graphics Processing Units (GPUs) which allows fast training of convolutional deep networks

This tutorial  is partly based on the offical tutorial that can be found [here](https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html)

Tensors
------

The central objects in Pytorch are tensors. Tensors are similar to NumPy’s ndarrays, with the addition being that
Tensors can also be used on a GPU to accelerate computing.

In [ ]:
from __future__ import print_function
import torch

Construct a 5x3 matrix, uninitialized:



In [ ]:
x = torch.empty(5, 3)
print(x)

Construct a randomly initialized matrix:



In [ ]:
x = torch.rand(5, 3)
print(x)

Construct a matrix filled with zeros or ones



In [ ]:
x = torch.zeros(5, 3)
y = torch.ones(5, 3)
print(x)
print(y)

Construct a tensor directly from a list:

In [ ]:
x = torch.tensor([5.5, 3])
print(x)

or create a tensor based on an existing tensor, here we use randn_like but zeros_like or ones_like work too.

In [ ]:
x = torch.empty(5, 3)
x = torch.randn_like(x, dtype=torch.float)
print(x)                                    

Get its size:



In [ ]:
print(x.size())

Operations on tensors
------

There are multiple syntaxes for operations. In the following
example, we will take a look at the addition operation.

Addition

In [ ]:
y = torch.rand(5, 3)
print(x + y)

Element-wise multiplication

In [ ]:
print(x*y)

Dot-products (dot) Matrix-vector products (mv)

In [ ]:
a = torch.rand(3)
b = torch.rand(3)
print(a.dot(b))
print(x.mv(a))

Sums of elements, (Euclidean) norm (square root of the sum of the squred elements of the tensor) 

In [ ]:
print(torch.sum(x))
print(torch.norm(a))

**Read later:**


  100+ Tensor operations, including transposing, indexing, slicing,
  mathematical operations, linear algebra, random numbers, etc.,
  are described
  here <https://pytorch.org/docs/torch>


You can use standard NumPy-like indexing with all bells and whistles!

In [ ]:
print(x[:, 1])

Resizing: If you want to resize/reshape tensor, you can use ``torch.view``:



In [ ]:
x = torch.randn(4, 4)
y = x.view(16)
z = x.view(-1, 8)  # the size -1 is inferred from other dimensions
print(x.size(), y.size(), z.size())

If you have a one element tensor, use ``.item()`` to get the value as a
Python number



In [ ]:
x = torch.randn(1)
print(x)
print(x.item())

In-place operations: Any operation that mutates a tensor in-place is post-fixed with an ``_``.
    For example: ``x.copy_(y)``, ``x.t_()``, will change ``x``



In [ ]:
# adds x to y
y.add_(x)
print(y)


CUDA Tensors
------------

Pytorch can handle gpu computations by using cuda if you have access to gpus (which is not the case in this tutorial). To handle gpu computations you need to transfer the data to the gpu devide as done below. Gpus are useful to get faster implementations of some operations such as convolutions of images. 

In [ ]:
# let us run this cell only if CUDA is available
# We will use ``torch.device`` objects to move tensors in and out of GPU
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    y = torch.ones_like(x, device=device)  # directly create a tensor on GPU
    x = x.to(device)                       # or just use strings ``.to("cuda")``
    z = x + y
    print(z)
    print(z.to("cpu", torch.double))       # ``.to`` can also change dtype together!

Automatic differentiation
=========

Follow the slides until slide 11. Below is a first scalar example

In [ ]:
# Instantiate variable with flag to record any computation involving w0
w0 = torch.rand(1, requires_grad = True)

# Compute logistic loss on (x,y) = (3.5, 1)
out = torch.log(1+torch.exp(-3.5*w0))

# Backpropagate gradients of any input involved in out
out.backward()

# Derivative is recorded in the variable
print(w0.grad)

Now with multivariate functions, slides 11-12

In [ ]:
# Create data n =100, d = 20
X = torch.rand(100, 20)

w0 = torch.rand(20, requires_grad=True)

# Compute logistic loss (mv stands for matrix vector product)
out = sum(torch.log(1+ torch.exp(-X.mv(w0))))

# Backpropagate gradients
out.backward()

print(w0.grad)
print(w0.grad.shape) # get 20 dimensional vector

Finally with two parameters introduced at different steps of the computations, slides 13-14

In [ ]:
# Instantiate weights
w1 = torch.rand(1, requires_grad = True); 
w2 = torch.rand(1, requires_grad = True)

# Compute square loss of 1-layer DNN with tanh activation on (x,y)=(3.5,1)
out = torch.tanh(3.5*w1)
out = w2*out
out = (1 - out)**2

# Backpropagate gradients of any input involved in out
out.backward()

# Gradients are recorded in the variables
print(w1.grad)
print(w2.grad)

Rather than using backward and having gradients stored implicitly in the variables, you can use torch.autograd.grad. This function directly returns the gradient.

In [ ]:
X = torch.rand(100, 20)

w0 = torch.rand(20, requires_grad=True)

out = sum(torch.log(1+ torch.exp(-X.mv(w0))))

# First argument is the result of the operations, 
# the second argument is the input from which the gradient is taken
# The second argument can take a list of inputs, the output of the function is then a tuple
# So in our case the output is a single tuple that we extract
grad = torch.autograd.grad(out, w0)[0]
print(grad)

Note that Pytorch only computes gradients of scalar functions. The reason is that one often does not need to compute the Jacobian matrix itself but the product of the Jacobian with any vector. 

Deep networks
=======
Formally, neural networks are composed of $L$ layers. 
The affine mapping at the $l^{\mathrm{th}}$ layer
is parameterized by the  weights-bias pair $u_l = (W_l, b_l)$. 
The complete mapping is then a function of $u = (u_1,\ldots,u_L)$  given by
$$
\Phi(x;u) = \phi_{u_{L}}\circ \phi_{u_{L-1}} \circ \ldots \circ \phi_{u_{1}}(x),
$$
where, for any $l = 1,\ldots, L$,
$$
\phi_{u_l}(x) =  \sigma(W_l x +b_l),
$$
and $\sigma$ is a non-linear operation such as the point-wise application of the 
soft-plus function $\log(1+\exp(\beta x))/\beta$.

On an input-output pair $(x,y)$, the error of prediction of $y$ by $\Phi(x;u)$ is measured by a loss $\ell(y, \Phi(x;u))$ such as the squared loss, $\ell_2(y,\Phi(x;u))=\frac{1}{2} \|y-\Phi(x;u)\|_2^2$ or logistic loss and the learning procedure consists of minimizing the empirical loss on the set of input-output training examples $(x_1,y_1),\ldots,(x_n,y_n)$ leading to the optimization problem
$$
   \min_{\substack{u = (u_1,\ldots,u_L)}} \quad \frac{1}{n} \sum_{i=1}^n \ell(y_i,\Phi(x_i;u)).
$$

Instantiate a simple network
----------------------------------
A neural network can easily be defined using the framework provided by [PyTorch](https://pytorch.org/). There, a network is an object defined by
* the affine transformations parameterized by $u_1,\ldots,u_L$ above, that are the attributes of the network , through the constructor `__init__`,
* the whole operations made on an input by the network, i.e. the function $\Phi$ defined above, (so composing the linear transformations with non-linear ones), through the function <code>forward</code>.

We will see different linear and non-linear operations in this lab. For the moment we will just use the ones presented before, i.e. affine transformation and the softplus function.

To begin with, we will define a simple Multi-Layer Perceptron (MLP) for images with two layers, i.e. parameterized by two linear transformations ($u_1, u_2$). The Mapping $\Phi$ will then reduce the input (in this case, an image) to a vector, apply the first affine map (such a layer is called fully connected (fc below)), then the soft-plus function as the non-linearity $\sigma$ and finally apply the second affine map.

Below is the code in PyTorch to define this MLP:

In [ ]:
from torch import nn
from torch.nn.functional import softplus
class MLPNet(nn.Module):
    """
    Simple multi-layer perceptron (MLP) with one hidden layer
    """
    def __init__(self, size_img, num_hidden_nodes, num_classes):
        super(MLPNet, self).__init__()
        # First linear map form the space of images represented by 
        # vectors of size size_img to the dimension of the first layer
        self.size_img = size_img
        self.fc1 = nn.Linear(size_img, num_hidden_nodes)
        # Second linear map from output of layer 1 to the output space (here composed of num_classes)
        self.fc2 = nn.Linear(num_hidden_nodes, num_classes)
        # Define the non-linearity
        # threshold parameter is properly defined in pytorch manual
        self.nonlin = lambda x: softplus(x, beta=25, threshold=1)
        # self.nonlin = relu # uncomment to use ReLu instead
    def forward(self, x):
        # Reduce image to a vector:
        x = x.view(-1, self.size_img)
        # Apply first linear map followed by the non-linearity
        x = self.nonlin(self.fc1(x))
        # Apply the second linear map
        x = self.fc2(x)
        return x

Data generation
---------------
We will use this network to classify handwritten digits from the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) (see examples of data [here](https://oriamathematics.wordpress.com/2016/08/12/binary-classification-with-logistic-regression/)). These are 28x28 images  of digits 0,...,9 (10 classes) represented by a one-hot encoding. That is, image $x_i \in \mathbb{R}^{28 \times 28}$ and label $y_i \in \{0, 1\}^{10}$. Note that $y_i$ is represented by a binary vector with the $k$-th entry equal to $1$ if the image $x_i$ belongs to class $k$ and $0$ otherwise. We define a network with 500 hidden nodes in the first layer and use the square loss. 
Formally, we require our neural network $\Phi : \mathbb{R}^{28 \times 28} \to \mathbb{R}^{10}$ to have 10 nodes in the last (output) layer. 

In [ ]:
# Instantiate neural network and loss
net = MLPNet(28*28, 500, 10)
loss_fn = nn.MSELoss() # MSE = mean squared error

The loading and the preprocessing of the data is handled by the library torchvisions that is provided in Pytorch. It gives access to a large library of datasets and preprocessing tools.

In [ ]:
# Set up dataset and dataloader see Pytorch documentation 
# <https://pytorch.org/tutorials/beginner/data_loading_tutorial.html> for more details
import torchvision
from torchvision import transforms
# Pre-computed mean and standard deviation.
mean_MNIST = 0.1307
std_MNIST = 0.3081
# Preprocess the data 
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((mean_MNIST,), (std_MNIST,))])
# Load MNIST training set with one hot encoding of labels 
# (i.e. y is normally given as a digit between 0 and 9 such as y=6 
# and we transform it into a vector of 10 coordinates with a one on the coordinate of the digit
# here a one on the 6th coordinate if y=6)
class MakeOneHot(object):
    """
    Transform ordinary output into corresponding binary vector
    """
    def __init__(self, nb_classes):
        self.nb_classes = nb_classes

    def __call__(self, target):
        out = torch.eye(self.nb_classes)
        return out[target].view(-1)

target_transform = MakeOneHot(10)
trainset = torchvision.datasets.MNIST(root='./data/MNIST', train=True, target_transform=target_transform,
                                      transform=transform, download=True)
# Dataloader will output batch of samples form the data set, randomly chosen by suffling the dataset 
# To load a batch of batch_size sample at each time we will use 
# dataloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=1)

## Optimization
We will now compare different optimization methods to solve the training problem that can be simply written as
$$
\min_u \, f(u) = \frac{1}{n}\sum_{i=1}^n f_i(u)
$$
where $f_i(u) = \ell(y_i,\Phi(x_i;u))$ is the loss encountered by the set of parameters on the ith training sample. 

We focus on first order algorithms that use gradients on the problem. To compute them, notice that each $f_i$ is a composition of functions, such that its gradient is given by the chain rule. Precisely, the gradient of $f_i$ with respect to the $l$th variables of the network $u_l$ will depend on the gradient of the next layers for $l' = l+1,\ldots,L$. 

Practically in the PyTorch framework, given a sample, the neural net records all the operations it computes from the input $x_i$ until the value of the loss with the parameters of the network. From these operations that it recorded (from the definition of the loss and the <code>forward</code> function above), it derives the gradient of the corresponding function $f_i$.

The following code computes the gradient of the network with respect to as many samples as it gets (in x, y below).

__Explanation__: 
- The various parameters of a neural network (net), represented by an object of type nn.Module may be accessed individually via their names, e.g. `net.fc1.weight` or `net.fc2.bias` in `MLPNet` above, or via the iterator `net.parameters()`. 
- The function below returns the loss, which can then be used for aggregation and book-keeping, and a reference to the gradient.

In [ ]:
def stoc_grad_(net, loss_fn, x, y):
    """
    The function computes gradient of `loss_fn(net(x), y)`
    w.r.t. parameters of `net`, where `x` is the input `y` is the label.
    Returns the function value `loss_fn(net(x), y)` and the gradient.
    """
    loss = loss_fn(net(x), y) # compute loss
    # Return reference to gradient with respect to all the parameters of the network
    # The parameters of the network correspond here to all affine maps defiend above.
    # The W and b of each affine layer is recorded when the network is initialized 
    # and added to the list of parameters that can be called like here 
    grad = torch.autograd.grad(loss, [param for param in net.parameters()])
    return loss, grad

### Stochastic Gradient Descent (SGD)
For large data sets, computing the whole gradient may be costly. Therefore one can rather use approximations of it: compute the gradient of a random subset $I$ of size $m$ of the samples, 
$$
\tilde \nabla f(u) = \frac{1}{m}\sum_{i \in I} \nabla f_i(u)
$$
By drawing the subsets at random, we ensure that this approximate gradient is a unbiased estimate of the true gradient, i.e. $E(\tilde \nabla f(u)) = \nabla f(u)$. Random subsets of more than one sample are called *mini-batches*. 

From this estimate of the gradient we can apply the stochastic gradient descent that starts from a given $u_0$ and follows the rule
$$
u_{k+1} = u_k - \gamma_k \tilde \nabla f(u_k) 
$$
where $\gamma_k$ is the step-size (learning rate).

The following function implements SGD with a constant learning rate `lr`. 

In [ ]:
import time
import numpy as np
def sgd(net, loss_fn, dataloader, lr, num_epochs):
    """
        run SGD with constant learning rate
        - For inputs x,y, the function is defined as `loss_fn(net(x), y)`.
        - `dataloader` is an iterator over the examples $\{x_i, y_i\}_{i=1}^n$.
        - `lr` is the (constant) learning rate $gamma$ used by SGD.
        - SGD is run for `num_epochs` number of passes through the dataset.
    """
    losses = []
    print('Epoch\tAvg loss\tTime')
    for epoch in range(num_epochs):
        t1 = time.time()
        for i, (images, labels) in enumerate(dataloader):
            # dataloader here gives randomly sampled images and labels
            loss, grad = stoc_grad_(net, loss_fn, images, labels)
            losses += [loss.item()]
            with torch.no_grad():
                for var, grad_var in zip(net.parameters(), grad):
                    # grad_var is a pointer to gradient of loss w.r.t. var
                    updated_var = var - lr*grad_var
                    var.copy_(updated_var) # copy updated_var into var
            if i % 100 == 99: # Logging
                print('{:.2f}\t{:.4f}\t\t{:.2f}'.format(
                    epoch + i / len(dataloader), # number of passes
                    np.asarray(losses).mean(),  # mean loss seen this epoch
                    time.time() - t1))
        losses = []

### Methodology to find hyper-parameters of stochastic gradient descent

This algorithm can be proven to converge to a stationary point under mild assumptions on the smoothness of the function. This depends obviously on the choice of the step-sizes that shall tend to zero to ensure the reduction of the variance of the estimates. Theory tells that a step-size of $\gamma_k \propto 1/\sqrt{k+1}$ is optimal, practitioners often use heuristics that are found to work better in practice.

Precisely, practitioners run stochastic gradient descent with a constant step-size $\gamma$ until a given stopping criterion, then they restart the algorithm from the final point using a new constant step-size $\delta \gamma$, reduced by a fixed factor $\delta < 1$, and repeat the process. This strategy requires three hyper-parameters: 
- initial learning rate $\gamma_0$, 
- stopping criterion,
- learning rate decay factor $\delta$.

Let us look at each in turn:
- __Initial learning rate $\gamma_0$__: The initial learning rate $\gamma_0$ must be large enough so that the algorithm makes good progress initially, but not so large that the optimization diverges. Too small an initial learning rate leads to very slow optimization. 
- __Stopping Criterion__: Two stopping criteria are commonly used: 
    - *fixed iteration budget*: Stop after a fixed number $T_{\mathrm{budget}}$ number of epochs. Here, $T_{\mathrm{budget}}$  is a hyper-parameter.
    - *plateau on a validation set*: Use a validation set to stop when the performance on it (either in loss value or classification accuracy)  has not improved in a fixed number $T_{\mathrm{patience}}$ number of epochs where $T_{\mathrm{patience}}$ is a hyper-parameter.
- __Decay factor $\delta$__: As for the learning rate, the decay factor $\delta$, shouldn't be too small in order to avoid to be quickly stuck.

Practitioners use default values for $T_{\mathrm{budget}}$ or $T_{\mathrm{patience}}$ and search only for the initial learning rate $\gamma_0$ and the decay factor $\delta$. We don't detail here the grid search for the decay factor, simply know that it is commonly searched on a grid $\delta \in\{1/2, 1/4, 1/8\}$.

####  Test SGD
Let us now try to experiment some constant learning rates for SGD. 
Intuitively very large learning rates cause the optimization to diverge
i.e. the function value and parameter norms quickly blow up to $+\infty$ (or you might notice NaNs that arise from arithmetic operation involving $\infty$).
Here we will find a learning rate $\gamma$ so that SGD with learning rate $c\gamma$, $c > 1$ (e.g., $10$), diverges 
immediately (i.e., within a few tens of updates)
but doesn't with a learning rate $\gamma$.

For the case of [cross entropy loss (log loss)](https://en.wikipedia.org/wiki/Loss_functions_for_classification#Cross_entropy_loss_(Log_Loss)),
which is popular for classification problems, 
too large a learning rate causes the function value to become large (although not $+\infty$)
and never drop below $\ln 10 \approx 2.3$. Note that the dataset MNIST
that we consider has 10 classes, and a function value of $\ln 10$ corresponds 
to random guessing where each class is assigned a probability of $1/10$.

**Exercise 1:**
Try to find a good initial learning rate up as follows: find a number $\gamma_0$ so that 
SGD works with an initial learning rate of $\gamma_0$ 
but diverges with $10\gamma_0$. 
Once we have this estimate, $\gamma_0$ or $\gamma_0 / 10$ are usually good candidates for initial learning rates.

**Exercise 2:**
Try different batch-sizes 50, 100, 200. Find the best learning rate for each batch-size. Compare the difference in the speed of the decreasing of the loss for the best learning rate you can find.

**Solution 1:**
You might have found above that $\gamma=0.1$ or $\gamma=1.0$ 
results in a rapid decrease of the loss while
$\gamma = 10.0$ 
causes NaNs, a sign of divergence.
A smaller step-size of $\gamma = 0.01$ also works but is much more conservative
than the choice $\gamma = 0.1$.

In [ ]:
batch_size=200
dataloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=1)
net = MLPNet(28*28, 500, 10) # Reinitialize the net when using a different learning rate
lr = 1e-1 # learning rate
sgd(net, loss_fn, dataloader, lr, num_epochs=2) # call SGD